<a href="https://colab.research.google.com/github/eflatlan/CNN_PID/blob/populateAll/cnn_adjusted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h5py numpy

import os
import h5py
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import random

def plot_random_element(X_train_map):
    index = random.randint(0, len(X_train_map) - 1)  # Pick a random index
    element = X_train_map[index, :, :, 0]  # Retrieve the element

    plt.figure(figsize=(8, 6))
    plt.imshow(element, cmap='viridis', origin='lower')
    plt.title(f"Random Element from X_train_map (Index {index})")
    plt.colorbar(label='Intensity')
    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.show()






In [3]:
def plot5(X_test_map, particle_vector):

  # Plotting random maps with information

  # Select 5 random indices from the test data
  random_indices = np.random.choice(range(X_test_map.shape[0]), size=5, replace=False)

  # Create a subplot with 5 rows and 1 column
  fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(8, 20))

  # Iterate over the random indices and plot each map with information
  for i, index in enumerate(random_indices):
      # Get the map and corresponding information
      map_data = X_test_map[index, :, :, 0]
      mass_category = particle_vector[index].mass_category
      ckov = particle_vector[index].ckov
      mip_position = particle_vector[index].mip_position
      momentum = particle_vector[index].momentum

      # Plot the map
      axes[i].imshow(map_data, cmap='gray')

      # Add a red dot at the MIP position
      axes[i].plot(mip_position[0], mip_position[1], 'ro')

      # Set the title with the information
      axes[i].set_title(f"Mass: {mass_category}, CKOV: {ckov}, MIP Position: {mip_position}, Momentum: {momentum}")
      axes[i].axis('off')

  # Adjust the spacing between subplots
  plt.tight_layout()

  # Show the plot
  plt.show()

In [4]:
def create_lr_scheduler(num_epochs = 10):

  tf.random.set_seed(42)
  div = num_epochs/4
  print("div =", div)
  print("1e-4 * 10**(epoch/div) = ", 1e-4 * 10**(num_epochs/div))
  lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/div)) # traverse a set of learning rate values starting from 1e-4, increasing by 10**(epoch/20) every epoch
  return lr_scheduler





def plot_lr(num_epochs = 10, history = None):
  div = num_epochs/4
  lrs = 1e-4 * (10 ** (np.arange(num_epochs)/div))
  plt.figure(figsize=(10, 7))
  plt.semilogx(lrs, history.history["loss"]) # we want the x-axis (learning rate) to be log scale
  plt.xlabel("Learning Rate")
  plt.ylabel("Loss")
  plt.title("Learning rate vs. loss");

In [14]:
class ParticleDataUtils:
    class Candidate2:
        def __init__(self, x, y, candStatus):
            self.x = x
            self.y = y
            self.candStatus = candStatus

    class ParticleInfo: # pls help me understand if any of these fields are missing from the X_train :
        def __init__(self, momentum, mass, energy, refractiveIndex, ckov, xRad, yRad, xPC, yPC, thetaP, phiP, arrayInfo, candsCombined):
            self.momentum = momentum # this dhould be with
            self.mass = mass    # not with
            self.energy = energy # with
            self.refractiveIndex = refractiveIndex # with
            self.ckov = ckov # not with
            self.xRad = xRad # with
            self.yRad = yRad # with
            self.xPC = xPC# with
            self.yPC = yPC# with
            self.thetaP = thetaP# with
            self.phiP = phiP# with
            self.arrayInfo = arrayInfo # do not include this to model
            self.candsCombined = candsCombined # with the field candStatus is a int that is 0..7, please make it categorical
            self.mass_category = self.infer_mass_category(mass)
            self.mip_position = [xPC, yPC]
            self.rad_position = [xRad, yRad]

        @staticmethod
        def infer_mass_category(mass):
            pion_mass = 0.1396
            proton_mass = 0.938
            kaon_mass = 0.4937

            if abs(mass - pion_mass) < 1e-6:
                return "pion"
            elif abs(mass - proton_mass) < 1e-6:
                return "proton"
            elif abs(mass - kaon_mass) < 1e-6:
                return "kaon"
            else:
                return "unknown"

def load_particle_info_from_hdf5(filename):
    particle_vector = []

    with h5py.File(filename, 'r') as file:
        first_group_name = list(file.keys())[0]
        second_group_name = list(file.keys())[1] # print these values?
        #second_group_name = list(file.keys())[1] # print these values?


        for f in first_group_name: print(f)

        group = file[first_group_name]

        print("Attributes for group '{}':".format(first_group_name))
        for attr_name, attr_value in group.attrs.items():
          print("{}: {}".format(attr_name, attr_value))


        print("Attributes for group '{}':".format(second_group_name))
        for attr_name, attr_value in group.attrs.items():
          print("{}: {}".format(attr_name, attr_value))

        for i, group_name in enumerate(file):
            group = file[group_name]

            momentum = group.attrs['Momentum']
            mass = group.attrs['Mass']
            energy = group.attrs['Energy']
            refractiveIndex = group.attrs['RefractiveIndex']
            ckov = group.attrs['Ckov']
            xRad = group.attrs['xRad']
            yRad = group.attrs['yRad']
            xPC = group.attrs['xPC']
            yPC = group.attrs['yPC']
            thetaP = group.attrs['ThetaP']
            phiP = group.attrs['PhiP']

            arrayInfo = [group.attrs['ArrayInfo0'], group.attrs['ArrayInfo1'], group.attrs['ArrayInfo2'], group.attrs['ArrayInfo3']]

            # Read arrayInfo
            # rrayInfo_dataset = group['ArrayInfo']
            # rrayInfo_data = arrayInfo_dataset[...]

            # Read candsCombined as complex type
            #candsCombined_dataset = group['candsCombined']

            candsCombined_dataset = group['candsCombined']

            candsCombined_data = candsCombined_dataset[...]
            #candsCombinedVec = [ParticleDataUtils.Candidate2(x['x'], x['y']) for x in candsCombined_data]
            # candsCombinedStatVec = [ParticleDataUtils.Candidate2(x['candStatus']) for x in candsCombined_data]
            #candsCombinedStatVec.to_list()
            #candsCombinedVec.to_list()

            candsCombined = [ParticleDataUtils.Candidate2(x['x'], x['y'], x['candStatus']) for x in candsCombined_data]

            particle_info = ParticleDataUtils.ParticleInfo(
                momentum, mass, energy, refractiveIndex, ckov, xRad, yRad, xPC, yPC, thetaP, phiP, arrayInfo = arrayInfo, candsCombined=candsCombined)

            particle_vector.append(particle_info)

    return particle_vector

def read_particle_data_from_file(filename="particle.h5"):
    drive_path = '/content/drive/MyDrive/Colab Notebooks/CERN_ML/CNN_PID/test/'  # Update the path to your Google Drive folder
    file_path = os.path.join(drive_path, filename)
    particle_vector = load_particle_info_from_hdf5(file_path)
    return particle_vector

# Prepare the training data
#X_train_candsCombined_xy = [np.array([(cand.x, cand.y) for cand in particle.candsCombined]) for particle in particle_vector]
#X_train_candsCombined_status = [to_categorical([cand.candStatus for cand in particle.candsCombined], num_classes=8) for particle in particle_vector]


# Prepare the additional training data
X_train_candsCombined_xy = []
X_train_candsCombined_status = []
# Extract other fields



filename = 'ParticleInfo.h5'
particle_vector = read_particle_data_from_file(filename)


for particle in particle_vector:
    cand_xy_list = []
    cand_status_list = []
    for cand in particle.candsCombined:
        cand_xy_list.append([cand.x, cand.y])
        cand_status_list.append(cand.candStatus)
        #print(cand.candStatus)
        #print(cand.x)
        #print("ca va")
    c = particle.refractiveIndex
    #print(c)
    #print((particle.candsCombined))
    #print(np.array(particle.candsCombined))

    #print(f"length of t {len(cand_xy_list)}")

    X_train_candsCombined_xy.append(cand_xy_list)
    X_train_candsCombined_status.append(cand_status_list)

# Convert them into NumPy arrays
X_train_candsCombined_xy = np.array(X_train_candsCombined_xy)
X_train_candsCombined_status = np.array(X_train_candsCombined_status)
# Normalize X_train_candsCombined_xy


from sklearn.preprocessing import StandardScaler, LabelBinarizer
std_scaler = StandardScaler()
label_binarizer = LabelBinarizer()

#X_train_candsCombined_xy = std_scaler.fit_transform(X_train_candsCombined_xy)

# One-hot encode X_train_candsCombined_status

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

X_train_candsCombined_status_encoded = mlb.fit_transform(X_train_candsCombined_status)

X_train_phi = [particle.phiP for particle in particle_vector]
X_train_theta = [particle.thetaP for particle in particle_vector]
X_train_energy = [particle.energy for particle in particle_vector]

# Convert to NumPy arrays
X_train_phi = np.array(X_train_phi)
X_train_theta = np.array(X_train_theta)
X_train_energy = np.array(X_train_energy)
X_train_candsCombined_xy = np.array(X_train_candsCombined_xy)
X_train_candsCombined_status = np.array(X_train_candsCombined_status)




# One-hot encode the labels
label_binarizer = LabelBinarizer()
y_train_encoded = label_binarizer.fit_transform([particle.mass_category for particle in particle_vector])
y_test_encoded = label_binarizer.transform([particle.mass_category for particle in particle_vector])
X_train_momentum = [particle.momentum for particle in particle_vector]


X_train_momentum = np.array(X_train_momentum)


X_train_mip_position = []
for i, particle in enumerate(particle_vector):
    # Reset the map_data for each particle
    #map_data = np.zeros(map_shape)
    X_train_mip_position.append(np.array(particle.mip_position))


X_train_mip_position = np.array(X_train_mip_position)


X_train_rad_position = []
for i, particle in enumerate(particle_vector):
    # Reset the map_data for each particle
    #map_data = np.zeros(map_shape)
    X_train_rad_position.append(np.array(particle.rad_position))

X_train_rad_position = np.array(X_train_rad_position)

# create a scaler object
std_scaler = StandardScaler()
# Normalize X_train_refractive_index
X_train_refractive_index = np.array([particle.refractiveIndex for particle in particle_vector])
X_train_refractive_index = X_train_refractive_index.reshape(-1, 1)
X_train_refractive_index = std_scaler.fit_transform(X_train_refractive_index)


from sklearn.model_selection import train_test_split
# Splitting the data into train and test sets
X_train_momentum, X_test_momentum, X_train_refractive_index, X_test_refractive_index, X_train_mip_position, X_test_mip_position, X_train_rad_position, X_test_rad_position, X_train_candsCombined_xy, X_test_candsCombined_xy, X_train_candsCombined_status, X_test_candsCombined_status, X_train_rad_position, X_test_rad_position, X_train_phi, X_test_phi, X_train_theta, X_test_theta, X_train_energy, X_test_energy, y_train, y_test = train_test_split(
    X_train_momentum,
    X_train_refractive_index,
    X_train_mip_position,
    X_train_rad_position,
    X_train_candsCombined_xy,
    X_train_candsCombined_status,
    X_train_rad_position,
    X_train_phi,
    X_train_theta,
    X_train_energy,
    y_train_encoded,
    test_size=0.2,
    random_state=42
)

from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_candsCombined_xy_padded = pad_sequences(X_train_candsCombined_xy, maxlen=100, padding='post')
X_train_candsCombined_xy_padded = np.array(X_train_candsCombined_xy_padded)

# do the same for the test set
X_test_candsCombined_xy_padded = pad_sequences(X_test_candsCombined_xy, maxlen=100, padding='post')
X_test_candsCombined_xy_padded = np.array(X_test_candsCombined_xy_padded)





X_train_candsCombined_status_padded = pad_sequences(X_train_candsCombined_status, maxlen=100, padding='post')
X_train_candsCombined_status_padded = X_train_candsCombined_status_padded[..., np.newaxis]  # Add a new dimension for features
X_test_candsCombined_status_padded = pad_sequences(X_test_candsCombined_status, maxlen=100, padding='post')
X_test_candsCombined_status_padded = X_test_candsCombined_status_padded[..., np.newaxis]  # Add a new dimension for features


candsCombined_xy_shape = (100, 2)  # if you choose to pad your sequences to length 100
candsCombined_status_shape = (100, 1)  # Now status is a sequence of length 100 with 1 feature



# Define the input shapes
momentum_shape = (1,)
refractive_index_shape = (1,)
mip_position_shape = (2,)
#candsCombined_xy_shape = X_train_candsCombined_xy.shape[1:]  # adjust as necessary
#candsCombined_status_shape = X_train_candsCombined_status.shape[1:]  # adjust as necessary
rad_position_shape = (1,)
phi_shape = (1,)
theta_shape = (1,)
energy_shape = (1,)


from __future__ import print_function
import os
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate, BatchNormalization, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import matplotlib.pyplot as plt


# Define inputs
# vectors, each element number n containing a x,y pair in  candsCombined_xy_input should be paired with the categorical value in
# candsCombined_status_input
candsCombined_xy_input = Input(shape=candsCombined_xy_shape, name='candsCombined_xy_input')
candsCombined_status_input = Input(shape=candsCombined_status_shape, name='candsCombined_status_input')

#scalars
momentum_input = Input(shape=momentum_shape, name='momentum_input')
refractive_index_input = Input(shape=refractive_index_shape, name='refractive_index_input')
phi_input = Input(shape=phi_shape, name='phi_input')
theta_input = Input(shape=theta_shape, name='theta_input')
energy_input = Input(shape=energy_shape, name='energy_input')

# x,y pairs
mip_position_input = Input(shape=mip_position_shape, name='mip_position_input')
rad_position_input = Input(shape=rad_position_shape, name='rad_position_input')



from tensorflow.keras.layers import Dense, Flatten, Input, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Define an input processing layer for candsCombined_xy
xy_dense = Dense(32, activation='relu')(candsCombined_xy_input)
xy_dense = Dropout(0.2)(xy_dense)

# Define an input processing layer for candsCombined_status
status_dense = Dense(32, activation='relu')(candsCombined_status_input)
status_dense = Dropout(0.2)(status_dense)

# Concatenate the xy and status layers
candsCombined_processed = concatenate([xy_dense, status_dense])

# Define a dense layer for the concatenated xy and status
candsCombined_dense = Dense(64, activation='relu')(candsCombined_processed)
candsCombined_dense = Dropout(0.2)(candsCombined_dense)

# Define the scalar inputs
scalar_inputs = concatenate([momentum_input, refractive_index_input, phi_input, theta_input, energy_input])

# Define a dense layer for the scalar inputs
scalar_dense = Dense(64, activation='relu')(scalar_inputs)
scalar_dense = Dropout(0.2)(scalar_dense)

# Define the position inputs
position_inputs = concatenate([mip_position_input, rad_position_input])

# Define a dense layer for the position inputs
position_dense = Dense(64, activation='relu')(position_inputs)
position_dense = Dropout(0.2)(position_dense)

# Concatenate all the processed inputs
from tensorflow.keras.layers import RepeatVector

# Define a dense layer for the scalar inputs
scalar_dense = Dense(64, activation='relu')(scalar_inputs)
scalar_dense = Dropout(0.2)(scalar_dense)
scalar_dense = RepeatVector(100)(scalar_dense)  # Repeat along the time dimension to match the shape of candsCombined

# Define a dense layer for the position inputs
position_dense = Dense(64, activation='relu')(position_inputs)
position_dense = Dropout(0.2)(position_dense)
position_dense = RepeatVector(100)(position_dense)  # Repeat along the time dimension to match the shape of candsCombined


concat = concatenate([candsCombined_dense, scalar_dense, position_dense])

# Define the final fully connected layers
fc1 = Dense(128, activation='relu')(concat)
fc1 = BatchNormalization()(fc1)
fc1 = Dropout(0.1)(fc1)

fc2 = Dense(32, activation='relu')(fc1)
fc2 = BatchNormalization()(fc2)
fc2 = Dropout(0.1)(fc2)

output = Dense(3, activation='softmax')(fc2)  # Predicting mass categories

# Define the model
model = Model(inputs=[candsCombined_xy_input, candsCombined_status_input, momentum_input, refractive_index_input, phi_input, theta_input, energy_input, mip_position_input, rad_position_input], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit([X_train_candsCombined_xy_padded, X_train_candsCombined_status_padded, X_train_momentum, X_train_refractive_index, X_train_phi, X_train_theta, X_train_energy, X_train_mip_position, X_train_rad_position],
                    y_train,
                    validation_data=([X_test_candsCombined_xy_padded, X_test_candsCombined_status_padded, X_test_momentum, X_test_refractive_index, X_test_phi, X_test_theta, X_test_energy, X_test_mip_position, X_test_rad_position], y_test),
                    epochs=10,
                    batch_size=32)




P
a
r
t
i
c
l
e
0
Attributes for group 'Particle0':
ArrayInfo0: 67.0
ArrayInfo1: 0.0
ArrayInfo2: 0.0
ArrayInfo3: 0.0
Ckov: 0.6438444256782532
Energy: 5.793657302856445
Mass: 0.4936999976634979
Momentum: 2.393141508102417
PhiP: -2.6928224563598633
RefractiveIndex: 1.2766509056091309
ThetaP: 0.1838526725769043
xPC: 14.573732376098633
xRad: 16.123476028442383
yPC: 130.95164489746094
yRad: 131.69790649414062
Attributes for group 'Particle1':
ArrayInfo0: 67.0
ArrayInfo1: 0.0
ArrayInfo2: 0.0
ArrayInfo3: 0.0
Ckov: 0.6438444256782532
Energy: 5.793657302856445
Mass: 0.4936999976634979
Momentum: 2.393141508102417
PhiP: -2.6928224563598633
RefractiveIndex: 1.2766509056091309
ThetaP: 0.1838526725769043
xPC: 14.573732376098633
xRad: 16.123476028442383
yPC: 130.95164489746094
yRad: 131.69790649414062


<ipython-input-14-15703af7f250>:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_candsCombined_xy = np.array(X_train_candsCombined_xy)
<ipython-input-14-15703af7f250>:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_candsCombined_status = np.array(X_train_candsCombined_status)


Epoch 1/10


ValueError: ignored

In [ ]:
# Plot training loss and validation loss
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()